In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from textwrap import wrap
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from IPython.display import Image, clear_output
import scipy.sparse
import math
np.random.seed(7)

# Load sequence data for each protein
all_seqs_df = pd.read_csv('./data/protein-structure/pdb_data_seq.csv')
# Load characteristic data for each protein
all_charcs_df = pd.read_csv('./data/protein-structure/pdb_data_no_dups.csv')

In [2]:
protein_charcs = all_charcs_df[all_charcs_df.macromoleculeType == 'Protein'].reset_index(drop=True)
protein_seqs = all_seqs_df[all_seqs_df.macromoleculeType == 'Protein'].reset_index(drop=True)

print(protein_charcs.head())
# print(protein_seqs.head())
# protein_df.isna().sum()
# protein_df.columns

  structureId         classification experimentalTechnique macromoleculeType  \
0        101M       OXYGEN TRANSPORT     X-RAY DIFFRACTION           Protein   
1        102L  HYDROLASE(O-GLYCOSYL)     X-RAY DIFFRACTION           Protein   
2        102M       OXYGEN TRANSPORT     X-RAY DIFFRACTION           Protein   
3        103L  HYDROLASE(O-GLYCOSYL)     X-RAY DIFFRACTION           Protein   
4        103M       OXYGEN TRANSPORT     X-RAY DIFFRACTION           Protein   

   residueCount  resolution  structureMolecularWeight crystallizationMethod  \
0           154        2.07                  18112.80                   NaN   
1           165        1.74                  18926.61                   NaN   
2           154        1.84                  18010.64                   NaN   
3           167        1.90                  19092.72                   NaN   
4           154        2.07                  18093.78                   NaN   

   crystallizationTempK  densityMatthews  de

In [3]:
protein_charcs = protein_charcs[['structureId','classification', 'residueCount', 'structureMolecularWeight',\
                         'crystallizationTempK', 'densityMatthews', 'densityPercentSol','phValue']]
protein_seqs = protein_seqs[['structureId','sequence']]

# combine protein characteristics df with their sequences using structureId
protein_all = protein_charcs.set_index('structureId').join(protein_seqs.set_index('structureId'))
protein_all = protein_all.dropna()

# capitalize all classification values to avoid missing any values in the next step
protein_all.classification = protein_all.classification.str.upper()

# drop all proteins with an unknown function; note -- the tilde returns the inverse of a filter
protein_all = protein_all[~protein_all.classification.str.contains("UNKNOWN FUNCTION")]

print(protein_all.head())

                            classification  residueCount  \
structureId                                                
1914                            ALU DOMAIN           232   
1A04           SIGNAL TRANSDUCTION PROTEIN           430   
1A04           SIGNAL TRANSDUCTION PROTEIN           430   
1A07         COMPLEX (TRANSFERASE/PEPTIDE)           222   
1A07         COMPLEX (TRANSFERASE/PEPTIDE)           222   

             structureMolecularWeight  crystallizationTempK  densityMatthews  \
structureId                                                                    
1914                         26562.73                 277.0             3.00   
1A04                         47657.25                 277.0             2.49   
1A04                         47657.25                 277.0             2.49   
1A07                         25718.97                 277.0             2.10   
1A07                         25718.97                 277.0             2.10   

             densi

In [4]:
class_count = protein_all.classification.value_counts()
functions = np.asarray(class_count[(class_count > 800)].index)
data = protein_all[protein_all.classification.isin(functions)]
data = data.drop_duplicates(subset=["classification","sequence"])  # leaving more rows results in duplciates / index related?
data.head()

,classification,residueCount,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,sequence
structureId,,,,,,,,
1A4S,OXIDOREDUCTASE,2012,217689.59,287.0,2.43,41.00,7.5,AQLVDSMPSASTGSVVVTDDLNYWGGRRIKSKDGATTEPVFEPATG...
1A6Q,HYDROLASE,382,42707.55,277.0,2.97,59.00,5.0,MGAFLDKPKMEKHNAQGQGNGLRYGLSSMQGWRVEMEDAHTAVIGL...
1A72,OXIDOREDUCTASE,374,40658.50,277.0,2.30,46.82,8.4,STAGKVIKCKAAVLWEEKKPFSIEEVEVAPPKAHEVRIKMVATGIC...
1A8O,VIRAL PROTEIN,70,8175.72,277.0,2.21,43.80,8.0,MDIRQGPKEPFRDYVDRFYKTLRAEQASQEVKNWMTETLLVQNANP...
1ACC,TOXIN,735,82849.97,277.0,2.30,47.00,6.0,EVKQENRLLNESESSSQGLLGYYFSDLNFQAPMVVTSSTTGDLSIP...


In [5]:
data.loc[~data['classification'].str.contains('ASE'), 'classification'] = 'OTHER'
data = data.loc[~data['classification'].str.contains("OTHER")]
data.loc[data['classification'].str.contains('TRANSFERASE/TRANSFERASE INHIBITOR'), 'classification'] = 'TRANSFERASE'
data.loc[data['classification'].str.contains('OXIDOREDUCTASE/OXIDOREDUCTASE INHIBITOR'), 'classification'] = 'OXIDOREDUCTASE'
data.loc[data['classification'].str.contains('HYDROLASE/HYDROLASE INHIBITOR'), 'classification'] = 'HYDROLASE'

print(data.classification.value_counts())
groups = np.asarray(data.classification.value_counts().index)

HYDROLASE         10056
TRANSFERASE        7391
OXIDOREDUCTASE     5332
LYASE              1939
ISOMERASE          1178
LIGASE             1067
Name: classification, dtype: int64


In [6]:
data

,classification,residueCount,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,sequence
structureId,,,,,,,,
1A4S,OXIDOREDUCTASE,2012,217689.59,287.0,2.43,41.00,7.5,AQLVDSMPSASTGSVVVTDDLNYWGGRRIKSKDGATTEPVFEPATG...
1A6Q,HYDROLASE,382,42707.55,277.0,2.97,59.00,5.0,MGAFLDKPKMEKHNAQGQGNGLRYGLSSMQGWRVEMEDAHTAVIGL...
1A72,OXIDOREDUCTASE,374,40658.50,277.0,2.30,46.82,8.4,STAGKVIKCKAAVLWEEKKPFSIEEVEVAPPKAHEVRIKMVATGIC...
1AE8,HYDROLASE,305,35880.88,277.0,2.66,53.69,7.0,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...
1AE8,HYDROLASE,305,35880.88,277.0,2.66,53.69,7.0,DFEEIPEEYL
...,...,...,...,...,...,...,...,...
6F6P,HYDROLASE,424,47994.95,291.0,2.61,56.00,7.3,GAASRLRSPSVLEVREKGYERLKEELAKAQRELKLKDEECERLSKV...
6F6P,HYDROLASE,424,47994.95,291.0,2.61,56.00,7.3,GASSRLRSPSVLEVREKGYERLKEELAKAQRELKLKDEECERLSKV...
6FAE,HYDROLASE,584,67448.08,293.0,2.26,45.62,8.0,MCKQTYQRETRHSWDSPAFNNDVVQRRHYRIGLNLFNKKPEKGIQY...


In [25]:
data=data.reset_index()

In [26]:
np.unique(data['classification'])

array(['HYDROLASE', 'ISOMERASE', 'LIGASE', 'LYASE', 'OXIDOREDUCTASE',
       'TRANSFERASE'], dtype=object)

In [27]:
y=data['classification']

In [28]:
X=data[['residueCount', 'structureMolecularWeight', 'crystallizationTempK', 'densityMatthews', 'densityPercentSol', 'phValue']]

In [29]:
import statsmodels.api as st
X = st.add_constant(X)

In [30]:
X

,const,residueCount,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue
0,1.0,2012,217689.59,287.0,2.43,41.00,7.5
1,1.0,382,42707.55,277.0,2.97,59.00,5.0
2,1.0,374,40658.50,277.0,2.30,46.82,8.4
3,1.0,305,35880.88,277.0,2.66,53.69,7.0
4,1.0,305,35880.88,277.0,2.66,53.69,7.0
...,...,...,...,...,...,...,...
26958,1.0,424,47994.95,291.0,2.61,56.00,7.3
26959,1.0,424,47994.95,291.0,2.61,56.00,7.3
26960,1.0,584,67448.08,293.0,2.26,45.62,8.0
26961,1.0,584,67448.08,293.0,2.26,45.62,8.0


In [32]:
mdl = st.MNLogit(y, X)
mdl_fit = mdl.fit(method='newton')
print (mdl_fit.summary())

Optimization terminated successfully.
         Current function value: 1.487728
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:         classification   No. Observations:                26963
Model:                        MNLogit   Df Residuals:                    26928
Method:                           MLE   Df Model:                           30
Date:                Sun, 01 Nov 2020   Pseudo R-squ.:                0.006187
Time:                        20:50:48   Log-Likelihood:                -40114.
converged:                       True   LL-Null:                       -40363.
Covariance Type:            nonrobust   LLR p-value:                 1.540e-86
     classification=ISOMERASE       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                            -4.0737      1.263     -3.226      0.

In [36]:
Xpred=mdl_fit.predict(X)

In [43]:
np.unique(Xpred.idxmax(axis=1),return_counts=True)

(array([0, 3, 4, 5]), array([26426,     1,   406,   130]))

In [47]:
confmatrix=mdl_fit.pred_table()

In [49]:
confmatrix

array([[9.805e+03, 0.000e+00, 0.000e+00, 0.000e+00, 2.300e+02, 2.100e+01],
       [1.174e+03, 0.000e+00, 0.000e+00, 0.000e+00, 3.000e+00, 1.000e+00],
       [1.042e+03, 0.000e+00, 0.000e+00, 0.000e+00, 2.200e+01, 3.000e+00],
       [1.928e+03, 0.000e+00, 0.000e+00, 0.000e+00, 5.000e+00, 6.000e+00],
       [5.197e+03, 0.000e+00, 0.000e+00, 0.000e+00, 7.200e+01, 6.300e+01],
       [7.280e+03, 0.000e+00, 0.000e+00, 1.000e+00, 7.400e+01, 3.600e+01]])

In [48]:
np.diag(confmatrix).sum()/confmatrix.sum()

0.36765196751103363